In [ ]:
# Import necessary modules
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
from sklearn import preprocessing
from sklearn.metrics import accuracy_score, recall_score, f1_score, confusion_matrix, ConfusionMatrixDisplay
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.inspection import permutation_importance

In [ ]:
# Display all columns
pd.set_option('display.max_columns', None)
# Customize column names
col_names = ['ID','Diagnosis','radius','texture','perimeter','area','smoothness','compactness',
             'concavity','concave_pts','symmetry','fractal_dim','radius_std','texture_std',
             'perimeter_std','area_std', 'smoothness_std','compactness_std','concavity_std',
             'concave_pts_std','symmetry_std','fractal_dim_std','radius_ext','texture_ext',
             'perimeter_ext','area_ext','smoothness_ext','compactness_ext','concavity_ext',
             'concave_pts_ext','symmetry_ext','fractal_dim_ext']
# Import data
df = pd.read_table('wdbc.data',sep=',',names = col_names)
df.head()

In [ ]:
# Import scaled data
scaled_df = pd.read_table('scaled_data',sep=',',index_col = 0)
scaled_df.head()

In [ ]:
# Examine distributions for each feature
num_fts = [i for i in num_cols if i != 'ID' and i != 'Diagnosis']
df.hist(column = num_fts, grid = False, xlabelsize = 6, layout = (9,3), figsize = (25,40));

In [ ]:
# Correlation matrix - separate matrix for means, standard errors, and extremes
fig,ax = plt.subplots(3, 1, figsize=(8,24))
fig.subplots_adjust(wspace = 8)

sns.heatmap(ax = ax[0], data = df[means].corr(numeric_only = True).abs(), fmt='.3f', annot = True)
ax[0].tick_params(labelrotation = 30)
sns.heatmap(ax = ax[1], data = df[stds].corr(numeric_only = True), fmt='.3f', annot = True)
ax[1].tick_params(labelrotation = 30)
sns.heatmap(ax = ax[2], data = df[exts].corr(numeric_only = True), fmt='.3f', annot = True)
ax[2].tick_params(labelrotation = 30)

ax[0].set_title('Correlation matrix for means')
ax[1].set_title('Correlation matrix for std errors')
ax[2].set_title('Correlation matrix for extremes')

plt.show();

Strongest correlations between radius/perimeter/area which makes sense geometrically. Area especially seems unnecessary since the original research actually says that area was just a function of a 2D area measurement + half perimeter.

Also strong correlations between compactness/concavity/concave points (and correlated with the three geometric features). Original research indicates that compactness is a combination (perimeter^2 / area) so can consider dropping a feature here.

Will be testing different feature subsets later on in modelling.

In [ ]:
# Creating a test subset (removing most highly correlated features) to view correlation
# between other features, then creating a new correlation matrix
subset01 = ['Diagnosis','texture','smoothness','symmetry','fractal_dim','texture_std',
           'smoothness_std','symmetry_std','fractal_dim_std','texture_ext','smoothness_ext',
           'symmetry_ext','fractal_dim_ext']
df_bool = df.loc[:, subset01]
df_bool['Diagnosis'] = df_bool['Diagnosis'].map({'M': 1, 'B': 0})

In [ ]:
fig,ax = plt.subplots(figsize = (8,6))
sns.heatmap(df_bool.corr(numeric_only = True).abs(), fmt='.2f', annot = True)
plt.title('Correlation matrix for test subset01')
plt.show();

Helps narrow down features for another possible subset - texture and texture_ext are highly correlated (the definition of the 'texture' measurement in the original research was vague). Same for smoothness, symmetry, and fractal dimension. Fractal dimension is also pretty highly correlated between mean and std error.

In [ ]:
# Plotting mean and std error correlations for 3 features
par_corrs = ['smoothness','symmetry','fractal_dim']

for f in par_corrs:
    ext = f + '_ext'
    sns.scatterplot(x = f, y = ext, alpha = .2, data = df, hue = 'Diagnosis', s = 20).set_title(f + ' parameters')
    plt.show()

Potential pattern with fractal dimension - negative (B) cases correlated with lower worst fractal dimension (as compared to means). The original research explains that fractal dimension is approximated using the slope (on a log scale) of coastline approximations of the perimeter, in decreasing precision (and therefore decreasing perimeter). Knowing this, it's surprising that fractal dimension and perimeter aren't more closely correlated. But it's plausible that contour irregularity might be fairly independent of actual perimeter measurements - for instance, nuclei with more extreme irregularity (concavity?) would shrink faster in a sequence of coastline approximations than nuclei with equal perimeter but less extreme irregularity (so probably larger overall).

In [ ]:
# Some collinearity is present - using a PCA to potentially determine another subset
pca = df.drop('ID', axis = 1).reset_index(drop = True).set_index('Diagnosis')
pca_idx = pca.index
pca_scal = pd.DataFrame(scale(pca), columns = pca.columns)
pca_scal.head()

In [ ]:
# Check scaling
pca_scal.std(ddof=0)

In [ ]:
# Fit PCA
pca_fit = PCA().fit(pca_scal)

In [ ]:
# Plot components vs. variance explained
plt.subplots(figsize = (10,5))

plt.plot(pca_fit.explained_variance_ratio_.cumsum())
plt.xlabel('Component')
plt.ylabel('Variance explained')
plt.title('Cumulative variance explained by PCA components')
plt.show();

In [ ]:
# Display actual percentages
variance = []

for comp in range(0,20):
    variance.append(str(comp) + ": " +
                    str(round(pca_fit.explained_variance_ratio_.cumsum()[comp], 4)))
display(variance)

It looks like the first 13 components explain over 98% of the variance in the data.

In [ ]:
# Extracting features from components - not precise, but useful as a possible subset for use
# in modelling later
comp_fts = pd.DataFrame(pca_fit.components_, columns = pca.columns).T

ft_counts = []

for i in range(0,14):
    fts = comp_fts.iloc[:, i].round(decimals = 3).sort_values(ascending=False)
    display(fts[0:2])
    ft_counts.append(fts[0:1].index)

In [ ]:
# Using Counter to find the most common features in the top 13 PCA components
top_fts = []

for i in range(0,14):
    top_fts.append(ft_counts[i][0])

Counter(top_fts).most_common

Concavity std seems to be a strong explainer of overall variance in the data. PCA suggests concavity, concave points, fractal dim, fractal dim std, texture std, smoothness std are important.

Interesting that standard error appears to be a good predictor? It could be useful for the model to know standard errors but seems like actual measurements should be better predictors. We will see how this subset does in modeling.

In [ ]:
# Quantiling each feature with .qcut - intervals are not equal but we can see how
# measurements are distributed for M (positive) cases
df_m = df.Diagnosis == 'M'

fig,axs = plt.subplots(9, 3, figsize = (30,1000))
fig.subplots_adjust(hspace = .8)
plt.rcParams['font.size'] = 8

for ft, ax in zip(paired_num_fts, axs.ravel()):
    qcats = pd.qcut(df[ft], q = [0, .2, .4, .6, .8, 1.],
                    labels = ['0% - 20%','20 - 40%','40 - 60%', '60 - 80%', '80 - 100%'],
                    duplicates = 'drop', precision = 1)
    qbins = np.array(qcats, dtype = object)
    ft_df = pd.DataFrame(list(zip(df_m.values, qbins))).groupby(1).mean()
    ft_df.plot(ax = ax, kind = 'bar', title = ft + ' vs. malignancy', legend = False,
              xlabel = 'Increasing ' + ft, ylabel = 'Proportion malignant', figsize = (12,25),
              rot = 45, fontsize = 6)

In [ ]:
# Dropping ID and setting Diagnosis to bool for RF (EDA) and modelling
tts_df = df.drop('ID', axis = 1)
tts_df['Diagnosis'] = tts_df['Diagnosis'].replace(to_replace = {'M': 1, 'B': 0})

# Splitting into train and test sets - will be the same as for modelling
X = clean_data.drop('Diagnosis', axis = 1)
y = clean_data.Diagnosis

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 17)

X_train.shape, y_train.shape, X_test.shape, y_test.shape

In [ ]:
# Random Forest - will look at feature importance, permutation importance (using train and
# validation sets from full train set only)
rf_i = RandomForestClassifier(random_state = 17, n_jobs = -1)
rf1 = rf_i.fit(X_train, y_train)

print(f"Accuracy on train set: {rf1.score(X_train, y_train):.4f}")
print(f"Accuracy on test set: {rf1.score(X_test, y_test):.4f}")

In [ ]:
# Display one of the trees
from sklearn.tree import export_graphviz
from subprocess import call
from IPython.display import Image

estimator = rf1.estimators_[5]

export_graphviz(estimator, out_file = 'tree_fi.dot', rounded = True, proportion = False,
               precision = 2, filled = True)
call(['dot', '-Tpng', 'tree_fi.dot', '-o', 'tree_fi.png', '-Gdpi=600'])
Image(filename = 'tree_fi.png')

In [ ]:
# Bar chart - feature importances
rf1_fi = rf1.feature_importances_
rf1_fi = 100 * (rf1_fi / rf1_fi.max())[:50]
idx = np.argsort(rf1_fi)[:50]

pos = np.arange(idx.shape[0]) + .5
plt.figure(figsize = (10,8))
plt.rcParams['font.size'] = 10
plt.barh(pos, rf1_fi[idx], align = 'center')
plt.yticks(pos, X.columns[idx])
plt.xlabel('Relative importance')
plt.title('RF feature importances')
plt.show();

Based on basic feature importance as calculated on the full training set, it seems that concave_pts_ext, perimeter_ext, concave_pts, radius_ext, area_ext, area, and concavity are strong predictors in RF.

Also running permutation importances - feature importances are based only on gini impurity and are biased toward high-cardinality (many distinct values) features, and also are computed only on the fitted model without validating performance against unseen data. Since the goal is to make a predictive model, I need to see which features are most important for helping the model generalize to unseen data.

The train/validation set used for permutation testing will be pulled from the actual train set and therefore will be a little smaller, but still sufficient - train set will be 64% of the original data and validation set will be 4%. At this point I will still hold out the actual test set so we can fairly assess model performance later on.

In [ ]:
X2 = X_train
y2 = y_train

X_trainb, X_testb, y_trainb, y_testb = train_test_split(X2, y2, test_size = .2,
                                                        random_state = 17)
rf2 = rf_i.fit(X_trainb, y_trainb)

print(f"Accuracy on train set: {rf2.score(X_trainb, y_trainb):.4f}")
print(f"Accuracy on test set: {rf2.score(X_testb, y_testb):.4f}")

Lower accuracy is expected - less training data provided.

In [ ]:
# Also checking recall and f1 scores
rf2_pred = rf2.predict(X_testb)
rf2_predproba = rf2.predict_proba(X_testb)[:,1]

rf2_r = recall_score(y_testb, rf2_pred)
rf2_f = f1_score(y_testb, rf2_pred, average = 'weighted')

rf2_cm = confusion_matrix(y_testb, rf2_pred)

print('\n Test recall:', round(rf2_r, 3), '\n Test f1:', round(rf2_f, 3))

In [ ]:
# Display confusion matrix
rf2_cm_disp = ConfusionMatrixDisplay(confusion_matrix = rf2_cm, display_labels = rf2.classes_)
rf2_cm_disp.plot(cmap = plt.cm.Blues)
plt.title('Confusion matrix, validation set RF')
plt.show()

In [ ]:
# Permutation importance to see if it confirms anything we saw in feature importance
# First running with f1 as scoring
result = permutation_importance(rf2, X_testb, y_testb, scoring = 'f1', n_repeats = 10, random_state = 17,
                               n_jobs = 2)

imp_sort_idx = result.importances_mean.argsort()

importances = pd.DataFrame(result.importances[imp_sort_idx].T, columns = X2.columns[imp_sort_idx])

# Boxplot
ax = importances.plot.box(vert = False, whis = 10, figsize = (10,10), grid = True)
ax.set_title("Permutation importances on f1")
ax.set_xlabel("Decrease in f1 score when permuted")
ax.figure.tight_layout()

In [ ]:
# Bar chart
fig,ax = plt.subplots(figsize = (10, 10))
plt.rcParams['font.size'] = 10

ax.barh(X_testb.columns[imp_sort_idx], result.importances[imp_sort_idx].mean(axis = 1))
ax.set_title('Permutation importances on f1')
ax.set_xlabel("Decrease in f1 score when permuted")
fig.tight_layout()

In [ ]:
# Now running with using roc_auc as scoring
result = permutation_importance(rf2, X_testb, y_testb, scoring = 'roc_auc', n_repeats = 10, random_state = 17,
                               n_jobs = 2)

imp_sort_idx = result.importances_mean.argsort()

importances = pd.DataFrame(result.importances[imp_sort_idx].T, columns = X2.columns[imp_sort_idx])

# Boxplot
ax = importances.plot.box(vert = False, whis = 10, figsize = (10,10), grid = True)
ax.set_title("Permutation importances on ROC-AUC")
ax.set_xlabel("Decrease in ROC-AUC score when permuted")
ax.figure.tight_layout()

In [ ]:
# Bar chart
fig,ax = plt.subplots(figsize = (10, 10))
plt.rcParams['font.size'] = 10

ax.barh(X_testb.columns[imp_sort_idx], result.importances[imp_sort_idx].mean(axis = 1))
ax.set_title('Permutation importances on ROC-AUC')
ax.set_xlabel("Decrease in ROC-AUC score when permuted")
fig.tight_layout()

Based on permutation testing, it seems that concave_pts_ext, texture, fractal_dim, fractal_dim_ext, smoothness_ext, symmetry_ext, and area are strong predictors.

concave_pts_ext, perimeter_ext, radius_ext, concave_pts, concavity, area_ext, radius, texture, perimeter, concavity_ext

So concave_pts_ext, perimeter_ext, radius_ext, concavity, radius, concavity_ext, and area are the predictors that are common in both feature and permutation importance with this split (both scoring methods).